In [1]:
from _element import feature_control as ft_c

In [2]:
from _usecase import algorithm_prophet as prpt

In [3]:
from _element import calculations as calc

In [4]:
import pandas as pd

In [5]:
from datetime import datetime

In [6]:
from _element import varr

In [7]:
INPUT_FILENAME= 'KPP일별_반납회수(13_17)_withobject.xlsx'

In [9]:
df_raw= ft_c.xlsx_opener(varr.PJ_DIR+varr.DF_DIR, INPUT_FILENAME)

In [10]:
df_temp= ft_c.xlsx_opener(varr.PJ_DIR+varr.TEMP_DATA_DIR, 'temp_data_merged.xlsx')

In [11]:
y_col= df_raw.columns.values.tolist()

In [12]:
try:
    y_col.remove('ds')
except ValueError:
    pass

In [13]:
y_col

[1041, 1692, 1815, 1853, 1878]

In [14]:
x_col= df_temp.columns.values.tolist()

In [15]:
x_col

['ds', 'temp_max', 'temp_min', 'rain_amount']

In [16]:
df_raw['ds']= pd.to_datetime(df_raw['ds'], box=True, format= '%Y/%m/%d', exact=True)

In [17]:
df_temp['ds']= pd.to_datetime(df_temp['ds'], box=True, format= '%Y/%m/%d', exact=True)

In [18]:
df_raw= pd.merge(df_raw, df_temp, how='inner', on='ds')

In [19]:
df_raw.head()

,ds,1041,1692,1815,1853,1878,temp_max,temp_min,rain_amount
0,2013-01-01,92,906,15693,3,7,-0.4,-12.900000,3.0
1,2013-01-02,1588,8278,43070,4410,2241,-9.4,-16.200001,0.0
2,2013-01-03,1944,7858,46003,6393,2967,-9.6,-15.300000,0.0
3,2013-01-04,2413,9612,46855,6838,3818,-6.2,-13.700000,0.0
4,2013-01-05,1337,7292,42439,4039,2354,-2.2,-10.900000,0.0


In [20]:
df_weekday_0= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==0]
df_weekday_1= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==1]
df_weekday_2= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==2]
df_weekday_3= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==3]
df_weekday_4= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==4]
df_weekday_5= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==5]
df_weekday_6= df_raw.loc[df_raw.ds.apply(lambda x: x.weekday())==6]

In [21]:
result_dict= {}
for y in y_col:
    result_dict[y]= {
                    'mon': prpt.Bayseian2(df_weekday_0, varr.FORECASTDAY, y, x_col, 'week'),\
                    'tue': prpt.Bayseian2(df_weekday_1, varr.FORECASTDAY, y, x_col, 'week'),\
                    'wed': prpt.Bayseian2(df_weekday_2, varr.FORECASTDAY, y, x_col, 'week'),\
                    'thu': prpt.Bayseian2(df_weekday_3, varr.FORECASTDAY, y, x_col, 'week'),\
                    'pri': prpt.Bayseian2(df_weekday_4, varr.FORECASTDAY, y, x_col, 'week'),\
                    'sat': prpt.Bayseian2(df_weekday_5, varr.FORECASTDAY, y, x_col, 'week'),\
                    'sun': prpt.Bayseian2(df_weekday_6, varr.FORECASTDAY, y, x_col, 'week'),\
                    }

INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling weekly seasonality. Run prophet with weekly_

In [22]:
merged_dict={}
for y in y_col:
    merged_dict[y]={'future': 
                    pd.concat([result_dict[y]['mon']['future'],\
                              result_dict[y]['tue']['future'],\
                              result_dict[y]['wed']['future'],\
                              result_dict[y]['thu']['future'],\
                              result_dict[y]['pri']['future'],\
                              result_dict[y]['sat']['future'],\
                              result_dict[y]['sun']['future']],\
                              axis= 0).sort_values(by='ds', axis=0),
                   'forecastProphetTable':
                    pd.concat([result_dict[y]['mon']['forecastProphetTable'],\
                              result_dict[y]['tue']['forecastProphetTable'],\
                              result_dict[y]['wed']['forecastProphetTable'],\
                              result_dict[y]['thu']['forecastProphetTable'],\
                              result_dict[y]['pri']['forecastProphetTable'],\
                              result_dict[y]['sat']['forecastProphetTable'],\
                              result_dict[y]['sun']['forecastProphetTable']],\
                              axis= 0).sort_values(by='ds', axis=0)
                   }

In [23]:
forecast_dict={}
print(INPUT_FILENAME)
for y in y_col:
    merged_dict[y]['future']['ds']= pd.to_datetime(merged_dict[y]['future']['ds'], box=True, format= '%Y/%m/%d', exact=True)
    merged_dict[y]['forecastProphetTable']['ds']= pd.to_datetime(merged_dict[y]['forecastProphetTable']['ds'], box=True, format= '%Y/%m/%d', exact=True)
    forecast_dict[y]= prpt.extract_info_from(merged_dict[y]['future'], merged_dict[y]['forecastProphetTable'], varr.FORECASTDAY)
    forecast_dict[y]['result_df']= pd.merge(forecast_dict[y]['result_df'], df_raw[['ds', y]], how='inner', on='ds')
    print('RMSE with segment '+str(y)+': '+str(calc.rms_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE with segment '+str(y)+': '+str(calc.map_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE_with_std with segment '+str(y)+': '+str(calc.map_error_with_std(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('MAPE_div_std with segment '+str(y)+': '+str(calc.map_error_div_std(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print('sMAPE of company code '+str(y)+': '+str(calc.smap_error(forecast_dict[y]['result_df'][y], forecast_dict[y]['result_df']['yhat'])))
    print(forecast_dict[y]['result_df'].head())
    ft_c.save_as_xlsx(varr.PJ_DIR+varr.DF_DIR, forecast_dict[y]['result_df'], INPUT_FILENAME, '_seg'+str(y)+'withweekday')
    print('\n')

KPP일별_반납회수(13_17)_withobject.xlsx
RMSE with segment 1041: 1071.39551737
MAPE with segment 1041: inf
MAPE_with_std with segment 1041: 44.5699241618
MAPE_div_std with segment 1041: 57.2767178689
sMAPE of company code 1041: 8.41020812269
          ds  temp_max  temp_min  rain_amount         yhat  1041
0 2017-11-20       3.7      -4.0          0.2  5019.239924  5619
1 2017-11-21       6.8       0.4          0.5  5509.996533  6057
2 2017-11-22       9.5      -0.8          0.0  5623.277575  5674
3 2017-11-23       4.3      -0.6          0.8  5438.740237  5686
4 2017-11-24       2.9      -2.5          0.1  5706.779431  6261


RMSE with segment 1692: 4959.89871444
MAPE with segment 1692: inf
MAPE_with_std with segment 1692: 35.6752437396
MAPE_div_std with segment 1692: 98.8183825702
sMAPE of company code 1692: 14.7352476342
          ds  temp_max  temp_min  rain_amount          yhat   1692
0 2017-11-20       3.7      -4.0          0.2  16711.308866  14471
1 2017-11-21       6.8       0.4      

C:\Nextop\nextop-engine\nextop_engine\_element\calculations.py:11: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100




RMSE with segment 1878: 1240.05707958
MAPE with segment 1878: inf
MAPE_with_std with segment 1878: 37.3691642755
MAPE_div_std with segment 1878: 62.4478182892
sMAPE of company code 1878: 7.90134722442
          ds  temp_max  temp_min  rain_amount         yhat  1878
0 2017-11-20       3.7      -4.0          0.2  5495.120153  5647
1 2017-11-21       6.8       0.4          0.5  5818.318164  6246
2 2017-11-22       9.5      -0.8          0.0  6016.332063  5510
3 2017-11-23       4.3      -0.6          0.8  6140.407778  5441
4 2017-11-24       2.9      -2.5          0.1  6783.237080  6290




In [23]:
# result_dict[y]['sun']['model'].plot(result_dict[y]['sun']['forecastProphetTable'])

In [24]:
# result_dict[y]['sun']['model'].plot_components(result_dict[y]['sun']['forecastProphetTable'])